# Modelo B 


por: Catalina Alvarez 

# Seccion 1: Setup del cuaderno

In [ ]:
import pandas as pd
#from ydata_profiling import ProfileReport

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud
#from num2words import num2words
import matplotlib.pyplot as plt
import re, string, unicodedata
import matplotlib

%matplotlib inline
import numpy as np
#import inflect
import joblib
import nltk

ModuleNotFoundError: No module named 'wordcloud'

# Sección 2 - Entendimiento de los datos 

In [ ]:
df_original = data = pd.read_csv('./datos/fake_news_spanish.csv', sep=';', encoding='utf-8', on_bad_lines='warn')
#df_test_original = pd.read_excel("./datos/fake_news_test.csv")


In [ ]:
df_original

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022


# Calidad de los datos

Dimensiones de calidad de los datos: 
    Consistencia 
    Validez 
    Campletitud 
    Unicidad 

# Completitud

Bajo una inspeccion sencilla, parece no haber problemas de completitud en los datos

In [ ]:
df_original["Label"].value_counts()

1    33158
0    23905
Name: Label, dtype: int64

In [ ]:
# strip all values in the column
df_original['Descripcion'] = df_original['Descripcion'].str.strip()
# count all values in the column where the value is an empty string

df_original['Descripcion'].isnull().sum()

0

# Validez 
Se identifican los siguientes problemas de validez: 
Los textos en el dataset tienen problemas, vocales con tilde han sido remplazados por caracteres no validos. 


In [ ]:
def fix_encoding(text):
    replacements = {
        'Ã¡': 'á',
        'Ã©': 'é',
        'Ã­': 'í',
        'Ã³': 'ó',
        'Ãº': 'ú',
        'Ã±': 'ñ'
    }
    
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    
    return text

df_original['Descripcion'] = df_original['Descripcion'].apply(fix_encoding)


In [ ]:
df_original.head(5)

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


# Consistencia 
Hay entradas entre comillas. Posiblemente esto no genere problemas en el dataset. 

# Unicidad
El dataset no tiene columanas que deben ser unicas. Bajo una sencilla inspeccion, hay filas duplicadas. 



In [ ]:
df_original.duplicated().sum()

445

In [ ]:
df_original.drop_duplicates(inplace=True)

In [ ]:
df_original.count()

ID             56618
Label          56618
Titulo         56602
Descripcion    56618
Fecha          56618
dtype: int64

In [22]:
def mostrar_grafico1(palabras,stopwords=[]):
    comment_words = ''
    
    # iterate through the csv file
    for val in palabras:
        
        # typecaste each val to string
        val = str(val)
    
        # split the value
        tokens = val.split()
        
        # Converts each token into lowercase
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()
        
        comment_words += " ".join(tokens)+" "
    
    
    wordcloud = WordCloud(width = 800, height = 800,
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10).generate(comment_words)
    
    # plot the WordCloud image                      
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout(pad = 0)
    
    plt.show()

In [23]:
stop_words = set(nltk.corpus.stopwords.words('spanish'))

for i in df_original['Label'].unique():
    print(f'---------- Words for class: {i} ----------')
    mostrar_grafico1(df_original.loc[df_original['Label']==i,'Titulo'], stopwords=stop_words)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Catalina/nltk_data'
    - 'c:\\Users\\Catalina\\anaconda3\\nltk_data'
    - 'c:\\Users\\Catalina\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\Catalina\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Catalina\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
